# Initialize Gamepad

In [17]:
class GamePadState:
    def __init__(self):
        self.all_profiles = {'Analog_sticks': 0, 'D_Pad': 1}
        self.profile = 'Analog_sticks'
        self.distance_x = 0
        self.distance_y = 0
        self.distance_z = 0
        self.distance_phi = 0
        self.step_size = 0.3
        self.step_size_phi = 0.3
        self.acceleration = 0.5

        self.buttons = {
        'home_x': 0,
        'home_y': 1,
        'home_z': 4,
        'home_phi': 3,
        'get_position': 11,
        'change_configuration': 6,
        'change_profile': 7,
        'change_step_size': 10,  
        'exit': 12
         }
        self.Axis = {
        'X': 0,
        'Y': 1,
        'Z':4,
        }
        self.D_Pad = {
        'phi' : 0,
        }
        print("GamePadState initialized in 'Fast' mode.")
        print("GamePadState initialized with 'Analog_sticks' profile.")
            
    def change_profile(self):
        if self.profile == 'Analog_sticks':
            self.profile = 'D_Pad'
            self.Axis = {
                'phi' : 0,
                'Z' : 4
            }
            self.D_Pad ={
                'XY' : 0
            }
            self.step_size = 0.03
            self.step_size_phi = 0.03
            
            print(f"Profile changed to {self.profile} mode.")
            
        elif self.profile == 'D_Pad':
            self.profile = 'Analog_sticks'
            self.Axis = {
            'X': 0,
            'Y': 1,
            'Z': 4,
            }
            self.D_Pad = {
            'phi' : 0,
            }
            self.step_size = 0.3
            self.step_size_phi = 0.3
            
            print(f"Profile changed to {self.profile} mode.")   
    
    def change_step_size(self):
        step_size = float(input("Enter desired stepsize: "))
        self.step_size = step_size
        self.step_size_phi = step_size
        print(f"Step size changed to {step_size}mm.")
        
        



    

In [ ]:
import pygame
import serial
import time

com_port = 'COM3'  # Adjust to your COM port
distance = 10  # Desired distance in mm for X axis
speed = 15  # Desired speed in mm/s for X axis
acceleration = 0.5  # Desired acceleration in mm/s^2 for X axis


def send_cmd(com_port, string1, display):
    ser = serial.Serial(port=com_port, baudrate=115200, parity=serial.PARITY_NONE,
                        stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS, timeout=0.1)
    to_send = string1.encode() + b'\r\n'
    ser.write(to_send)
    time.sleep(0.5)  # Delay for response
    response = ser.read(ser.in_waiting or 1).decode('utf-8')
    num = len(response)
    if num == 0:
        print('Timeout was reached.')
    elif display:
        print(f'String sent to Device: "{string1}"')
        print(f'Response from Device (in HEX): "{response.strip()}"')
        print(f'Number of Chars received: {num - 2} (+2 terminator chars)')
    ser.close()
    return response, num

def add_error_code(string):
    if len(string) % 2 == 0:
        print('Error: Input string length must be odd.')
        return -1
    temp = 0x100000000  # Equivalent to hex2dec('ffffffff') + 1
    string = string[1:]  # Remove the first character
    for i in range(0, len(string) - 1, 2):
        temp -= int(string[i:i+2], 16)
    temp_hex = '{:X}'.format(temp)
    complete_string = ':' + string + temp_hex[-2:]
    return complete_string


def home_x(com_port, gamepad):
    # Switch Modbus 'x' on
    command = add_error_code(':03050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on for X axis
    command = add_error_code(':03050403FF00')
    send_cmd(com_port, command, True)

    # Home X axis
    command = add_error_code(':0305040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command for X axis
    command = add_error_code(':0305040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':030390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F0" in response:
            print("Homing process for X axis completed successfully.")
            homing_completed = True
            gamepad.distance_x = 0
        else:
            print("Waiting for X axis homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1

def home_y(com_port, gamepad):
    # Switch Modbus 'x' on
    command = add_error_code(':02050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on for X axis
    command = add_error_code(':02050403FF00')
    send_cmd(com_port, command, True)

    # Home X axis
    command = add_error_code(':0205040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command for X axis
    command = add_error_code(':0205040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':020390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F1" in response:
            print("Homing process for X axis completed successfully.")
            homing_completed = True
            gamepad.distance_y = 0
        else:
            print("Waiting for X axis homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1


def home_z(com_port, gamepad):
    # Switch Modbus 'z' on
    command = add_error_code(':01050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on
    command = add_error_code(':01050403FF00')
    send_cmd(com_port, command, True)

    # Home Device
    command = add_error_code(':0105040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command
    command = add_error_code(':0105040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':010390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F2" in response:
            print("Homing process completed successfully.")
            homing_completed = True
            gamepad.distance_z = 0
        else:
            print("Waiting for homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1

def home_phi(com_port, gamepad):
    # Switch Modbus 'x' on
    command = add_error_code(':04050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on for X axis
    command = add_error_code(':04050403FF00')
    send_cmd(com_port, command, True)

    # Home X axis
    command = add_error_code(':0405040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command for X axis
    command = add_error_code(':0405040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':040390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98EF" in response:
            print("Homing process for Phi axis completed successfully.")
            homing_completed = True
            gamepad.distance_phi = 0
        else:
            print("Waiting for Phi axis homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1

def get_position_phi(com_port):
    # Send the command to get the X position
    command = add_error_code(':040390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    print(f"Current phi position: {position}mm")
    return position

def get_position_z(com_port):
    # Send the command to get the Z position
    command = add_error_code(':010390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    # Adjusted index for Python string and multiplied by 2 since each hex digit represents half a byte
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    print(f"Current Z position: {position}mm")
    
    return position

def get_position_y(com_port):
    # Send the command to get the X position
    command = add_error_code(':020390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    print(f"Current Y position: {position}mm")
    return position

def get_position_x(com_port):
    # Send the command to get the X position
    command = add_error_code(':030390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    print(f"Current X position: {position}mm")
    return position

def format_hex(value, length):
    """Format the given value as a hexadecimal string of the specified length."""
    return f'{value:0{length}X}'

def calculate_checksum(command):
    """Calculate the two's complement checksum for the given command."""
    sum_bytes = sum(int(command[i:i+2], 16) for i in range(1, len(command), 2))  # Start from 1 to skip ':'
    checksum = (0x10000 - sum_bytes) & 0xFFFF
    return format_hex(checksum, 4)[-2:]



def move_XY_plane(joystick, gamepad, com_port):
#in this function, we will move the XY plane based on the joystick input, and scale the distance depending on how much the joystick is pushed
 
    # Calculate the euclidean distance of the joystick input to scale speed depending on amout of joystick push
    euclidean_distance = (joystick.get_axis(gamepad.Axis['X'])**2 + joystick.get_axis(gamepad.Axis['Y'])**2)**0.5
    
    gamepad.distance_x += -1*joystick.get_axis(gamepad.Axis['X']) * gamepad.step_size
    gamepad.distance_y += joystick.get_axis(gamepad.Axis['Y']) * gamepad.step_size

    gamepad.distance_x = max(0, gamepad.distance_x)
    gamepad.distance_y = max(0, gamepad.distance_y)

    # Calculate the speed in X and Y directions based on the joystick input, to have smooth movement when moving diagonally
    if joystick.get_axis(gamepad.Axis['X']) > joystick.get_axis(gamepad.Axis['Y']) and joystick.get_axis(gamepad.Axis['X']) != 0:
        ratio = joystick.get_axis(gamepad.Axis['Y']) / joystick.get_axis(gamepad.Axis['X'])
        speed_x = gamepad.step_size * euclidean_distance
        speed_y = gamepad.step_size * euclidean_distance * ratio
    if joystick.get_axis(gamepad.Axis['X']) < joystick.get_axis(gamepad.Axis['Y']) and joystick.get_axis(gamepad.Axis['Y']) != 0:
        ratio = joystick.get_axis(gamepad.Axis['X']) / joystick.get_axis(gamepad.Axis['Y'])
        speed_x = gamepad.step_size * euclidean_distance * ratio
        speed_y = gamepad.step_size * euclidean_distance
    else:
        speed_x = gamepad.step_size * euclidean_distance
        speed_y = gamepad.step_size * euclidean_distance

    acceleration_x = gamepad.acceleration
    acceleration_y = gamepad.acceleration

    distance_hex_x = format_hex(int(gamepad.distance_x * 100), 8)
    distance_hex_y = format_hex(int(gamepad.distance_y * 100), 8)

    speed_hex_x = format_hex(int(speed_x * 100), 8)
    speed_hex_y = format_hex(int(speed_y * 100), 8)

    acceleration_hex_x = format_hex(int(acceleration_x * 100), 4)
    acceleration_hex_y = format_hex(int(acceleration_y * 100), 4)

    command_x = f'03109900000912{distance_hex_x}00000001{speed_hex_x[:4]}{speed_hex_x[4:]}{acceleration_hex_x}00000000'
    command_y = f'02109900000912{distance_hex_y}00000001{speed_hex_y[:4]}{speed_hex_y[4:]}{acceleration_hex_y}00000000'

    complete_command_x = f':{command_x}'
    complete_command_y = f':{command_y}'

    checksum_x = calculate_checksum(complete_command_x)
    checksum_y = calculate_checksum(complete_command_y)

    complete_command_with_checksum_x = f'{complete_command_x}{checksum_x}'
    complete_command_with_checksum_y = f'{complete_command_y}{checksum_y}'

    print(f"Constructed command for sending X: {complete_command_with_checksum_x}")
    print(f"Constructed command for sending Y: {complete_command_with_checksum_y}")

    send_cmd(com_port, complete_command_with_checksum_x, True)
    send_cmd(com_port, complete_command_with_checksum_y, True)
    
    if gamepad.step_size < 0.3:
        time.sleep(0.1)
    elif gamepad.step_size >= 0.3 and gamepad.step_size < 1:
        time.sleep(0.3)
    else:
        time.sleep(0.8)

def get_sign(num):
    if num > 0:
        return 1
    elif num < 0:
        return -1
    else:
        return 0

def move_z(joystick, gamepad, com_port):

    gamepad.distance_z += get_sign(joystick.get_axis(gamepad.Axis['Z'])) * gamepad.step_size
    gamepad.distance_z = max(0, gamepad.distance_z)

    speed_z = gamepad.step_size
    acceleration_z = gamepad.acceleration

    distance_hex_z = format_hex(int(gamepad.distance_z * 100), 8)
    speed_hex_z = format_hex(int(speed_z * 100), 8)
    acceleration_hex_z = format_hex(int(acceleration_z * 100), 4)

    command_z = f'01109900000912{distance_hex_z}00000001{speed_hex_z[:4]}{speed_hex_z[4:]}{acceleration_hex_z}00000000'

    complete_command_z = f':{command_z}'
    checksum_z = calculate_checksum(complete_command_z)
    complete_command_with_checksum_z = f'{complete_command_z}{checksum_z}'

    print(f"Constructed command for sending Z: {complete_command_with_checksum_z}")

    # Send the constructed command to the device
    send_cmd(com_port, complete_command_with_checksum_z, True)
    if gamepad.step_size < 0.3:
        time.sleep(0.1)
    elif gamepad.step_size >= 0.3 and gamepad.step_size < 1:
        time.sleep(0.3)
    else:
        time.sleep(0.5)

def move_phi(joystick, com_port, gamepad):

    gamepad.distance_phi += joystick.get_hat(gamepad.D_Pad['phi'])[0] * gamepad.step_size_phi
    if gamepad.distance_phi < 0:
        gamepad.distance_phi = 360 + gamepad.distance_phi
    elif gamepad.distance_phi >= 360:
        gamepad.distance_phi = gamepad.distance_phi - 360

        

    speed_phi = gamepad.step_size_phi
    acceleration_phi = gamepad.acceleration

    distance_hex_phi = format_hex(int(gamepad.distance_phi * 100), 8)
    speed_hex_phi = format_hex(int(speed_phi * 100), 8)
    acceleration_hex_phi = format_hex(int(acceleration_phi * 100), 4)

    command_phi = f'04109900000912{distance_hex_phi}00000001{speed_hex_phi[:4]}{speed_hex_phi[4:]}{acceleration_hex_phi}00000000'

    complete_command_phi = f':{command_phi}'
    checksum_phi = calculate_checksum(complete_command_phi)
    complete_command_with_checksum_phi = f'{complete_command_phi}{checksum_phi}'

    print(f"Constructed command for sending Phi: {complete_command_with_checksum_phi}")
    print(f"Virtual Phi position: {gamepad.distance_phi} degrees")

    # Send the constructed command to the device
    send_cmd(com_port, complete_command_with_checksum_phi, True)

def move_X_Dpad(joystick, gamepad, com_port):
    gamepad.distance_x += -1*joystick.get_hat(gamepad.D_Pad['XY'])[0] * gamepad.step_size
    
    gamepad.distance_x = max(0, gamepad.distance_x)

    speed_x = gamepad.step_size
    acceleration_x = gamepad.acceleration

    distance_hex_x = format_hex(int(gamepad.distance_x * 100), 8)

    speed_hex_x = format_hex(int(speed_x * 100), 8)

    acceleration_hex_x = format_hex(int(acceleration_x * 100), 4)

    command_x = f'03109900000912{distance_hex_x}00000001{speed_hex_x[:4]}{speed_hex_x[4:]}{acceleration_hex_x}00000000'

    complete_command_x = f':{command_x}'

    checksum_x = calculate_checksum(complete_command_x)

    complete_command_with_checksum_x = f'{complete_command_x}{checksum_x}'

    print(f"Constructed command for sending X: {complete_command_with_checksum_x}")

    send_cmd(com_port, complete_command_with_checksum_x, True)



def move_Y_Dpad(joystick, gamepad, com_port):
    gamepad.distance_y += -1*joystick.get_hat(gamepad.D_Pad['XY'])[1] * gamepad.step_size
    
    gamepad.distance_y = max(0, gamepad.distance_y)

    speed_y = gamepad.step_size
    acceleration_y = gamepad.acceleration

    distance_hex_y = format_hex(int(gamepad.distance_y * 100), 8)

    speed_hex_y = format_hex(int(speed_y * 100), 8)

    acceleration_hex_y = format_hex(int(acceleration_y * 100), 4)

    command_y = f'02109900000912{distance_hex_y}00000001{speed_hex_y[:4]}{speed_hex_y[4:]}{acceleration_hex_y}00000000'

    complete_command_y = f':{command_y}'

    checksum_y = calculate_checksum(complete_command_y)

    complete_command_with_checksum_y = f'{complete_command_y}{checksum_y}'

    print(f"Constructed command for sending Y: {complete_command_with_checksum_y}")

    send_cmd(com_port, complete_command_with_checksum_y, True)

def move_phi_analog(joystick, com_port, gamepad):

    #gamepad.distance_phi += get_sign(joystick.get_axis(gamepad.Axis['phi'])) * gamepad.step_size_phi
    #if gamepad.distance_phi < 0:
    #    gamepad.distance_phi = 360 + gamepad.distance_phi
    #elif gamepad.distance_phi >= 360:
    #    gamepad.distance_phi = gamepad.distance_phi - 360
    gamepad.distance_phi += joystick.get_hat(gamepad.D_Pad['phi'])[0] * gamepad.step_size
    
    gamepad.distance_phi = max(0, gamepad.distance_phi)

    speed_phi = gamepad.step_size_phi
    acceleration_phi = gamepad.acceleration

    distance_hex_phi = format_hex(int(gamepad.distance_phi * 100), 8)
    speed_hex_phi = format_hex(int(speed_phi * 100), 8)
    acceleration_hex_phi = format_hex(int(acceleration_phi * 100), 4)

    command_phi = f'04109900000912{distance_hex_phi}00000001{speed_hex_phi[:4]}{speed_hex_phi[4:]}{acceleration_hex_phi}00000000'

    complete_command_phi = f':{command_phi}'
    checksum_phi = calculate_checksum(complete_command_phi)
    complete_command_with_checksum_phi = f'{complete_command_phi}{checksum_phi}'

    print(f"Constructed command for sending Phi: {complete_command_with_checksum_phi}")
    print(f"Virtual Phi position: {gamepad.distance_phi} degrees")

    # Send the constructed command to the device
    send_cmd(com_port, complete_command_with_checksum_phi, True)

def initialize_gamepad():
    pygame.init()
    pygame.joystick.init()
    joystick_count = pygame.joystick.get_count()
    if joystick_count == 0:
        print("No gamepad connected")
        return None
    else:
        joystick = pygame.joystick.Joystick(0)
        joystick.init()
        print(f"Gamepad connected: {joystick.get_name()}")
        return joystick

def poll_gamepad(joystick, com_port):
    # Updated button mappings to include phi controls
  
    gamepad = GamePadState()

    running = True
    while running:
        pygame.event.pump()
        # Include checks and actions for X, Y, Z stages as before
        if gamepad.profile == 'Analog_sticks':
            if abs(joystick.get_axis(gamepad.Axis['X'])) >= 0.01 or abs(joystick.get_axis(gamepad.Axis['Y'])) >= 0.01:
                print("Moving in XY plane...")
                move_XY_plane(joystick, gamepad, com_port)
                
    
            if abs(joystick.get_axis(gamepad.Axis['Z'])) >= 0.01:
                print("Moving in Z axis...")
                move_z(joystick, gamepad, com_port)
    
            if joystick.get_hat(gamepad.D_Pad['phi']) != (0, 0):
                print("Moving in Phi axis...")
                move_phi(joystick, com_port, gamepad)
                
            if joystick.get_button(gamepad.buttons['home_x']):
                print("Homing X axis...")
                home_x(com_port, gamepad)
                time.sleep(0.5)
    
    
            if joystick.get_button(gamepad.buttons['home_y']):
                print("Homing Y axis...")
                home_y(com_port, gamepad)
                time.sleep(0.5)
    
    
            if joystick.get_button(gamepad.buttons['home_z']):
                print("Homing Z axis...")
                home_z(com_port, gamepad)
                time.sleep(0.5)
            
            if joystick.get_button(gamepad.buttons['home_phi']):
                print("Homing Phi axis...")
                home_phi(com_port, gamepad)
                time.sleep(0.5)
    
            if joystick.get_button(gamepad.buttons['get_position']):
                print("Getting current positions...")
                get_position_x(com_port)
                print("Virtual X position: ", gamepad.distance_x)
                get_position_y(com_port)
                print("Virtual Y position: ", gamepad.distance_y)
                get_position_z(com_port)
                print("Virtual Z position: ", gamepad.distance_z)
                get_position_phi(com_port)
                print("Virtual Phi position: ", gamepad.distance_phi)
                time.sleep(0.5)
    
            if joystick.get_button(gamepad.buttons['change_configuration']):
                gamepad.change_configuration()
                time.sleep(0.2)

            if joystick.get_button(gamepad.buttons['change_profile']):
                gamepad.change_profile()
                time.sleep(0.2)

            if joystick.get_button(gamepad.buttons['change_step_size']):
                gamepad.change_step_size()
                time.sleep(0.1)
    
    
            if joystick.get_button(gamepad.buttons['exit']):
                print("Exiting program...")
                running = False

        elif gamepad.profile == 'D_Pad': 

            if joystick.get_hat(gamepad.D_Pad['XY'])[0] != 0 or joystick.get_hat(gamepad.D_Pad['XY'])[1] != 0:
                print("Moving in X plane...")
                move_X_Dpad(joystick, gamepad, com_port)
                print("Moving in Y plane...")
                move_Y_Dpad(joystick, gamepad, com_port)

                if gamepad.step_size < 0.3:
                    time.sleep(0.1)
                elif gamepad.step_size >= 0.3 and gamepad.step_size < 1:
                    time.sleep(0.3)
                else:
                    time.sleep(0.5)
                
    
            if abs(joystick.get_axis(gamepad.Axis['Z'])) >= 0.01:
                print("Moving in Z axis...")
                move_z(joystick, gamepad, com_port)
    
            if abs(joystick.get_axis(gamepad.Axis['phi'])) >= 0.01:
                print("Moving in Phi axis...")
                move_phi_analog(joystick, com_port, gamepad)
                
            if joystick.get_button(gamepad.buttons['home_x']):
                print("Homing X axis...")
                home_x(com_port, gamepad)
                time.sleep(0.5)
       
            if joystick.get_button(gamepad.buttons['home_y']):
                print("Homing Y axis...")
                home_y(com_port, gamepad)
                time.sleep(0.5)
    
            if joystick.get_button(gamepad.buttons['home_z']):
                print("Homing Z axis...")
                home_z(com_port, gamepad)
                time.sleep(0.5)
            
            if joystick.get_button(gamepad.buttons['home_phi']):
                print("Homing Phi axis...")
                home_phi(com_port, gamepad)
                time.sleep(0.5)
    
            if joystick.get_button(gamepad.buttons['get_position']):
                print("Getting current positions...")
                get_position_x(com_port)
                print("Virtual X position: ", gamepad.distance_x)
                get_position_y(com_port)
                print("Virtual Y position: ", gamepad.distance_y)
                get_position_z(com_port)
                print("Virtual Z position: ", gamepad.distance_z)
                get_position_phi(com_port)
                print("Virtual Phi position: ", gamepad.distance_phi)
                time.sleep(0.5)
    
            if joystick.get_button(gamepad.buttons['change_configuration']):
                gamepad.change_configuration()
                time.sleep(0.2)

            if joystick.get_button(gamepad.buttons['change_profile']):
                gamepad.change_profile()
                time.sleep(0.2)

            if joystick.get_button(gamepad.buttons['change_step_size']):
                gamepad.change_step_size()
                time.sleep(0.1)
    
    
            if joystick.get_button(gamepad.buttons['exit']):
                print("Exiting program...")
                running = False    
            

        

def main():
    com_port = 'COM3'  # Adjust to your COM port

    joystick = initialize_gamepad()
    if joystick is not None:
        poll_gamepad(joystick, com_port)
    pygame.quit()

if __name__ == "__main__":
    main()

In [ ]:

def move_X_Dpad(gamepad, com_port):
    gamepad.distance_x += 1 * gamepad.step_size
    
    gamepad.distance_x = max(0, gamepad.distance_x)

    speed_x = gamepad.step_size
    acceleration_x = gamepad.acceleration

    distance_hex_x = format_hex(int(gamepad.distance_x * 100), 8)

    speed_hex_x = format_hex(int(speed_x * 100), 8)

    acceleration_hex_x = format_hex(int(acceleration_x * 100), 4)

    command_x = f'03109900000912{distance_hex_x}00000001{speed_hex_x[:4]}{speed_hex_x[4:]}{acceleration_hex_x}00000000'

    complete_command_x = f':{command_x}'

    checksum_x = calculate_checksum(complete_command_x)

    complete_command_with_checksum_x = f'{complete_command_x}{checksum_x}'

    print(f"Constructed command for sending X: {complete_command_with_checksum_x}")

    response = send_cmd(com_port, complete_command_with_checksum_x, True)
    print("Response from Device: ", response)

def main():
    com_port = 'COM3'  # Adjust to your COM port
    gamepad = GamePadState()

    
    print("Moving in X plane...")
    move_X_Dpad(gamepad, com_port)


if __name__ == "__main__":
    main()

# Check Gamepad Mapping

In [1]:
import pygame

def initialize_pygame():
    pygame.init()
    pygame.joystick.init()
    print(f"Detected {pygame.joystick.get_count()} joystick(s)")

def check_controllers():
    joystick_count = pygame.joystick.get_count()
    for i in range(joystick_count):
        joystick = pygame.joystick.Joystick(i)
        joystick.init()
        print(f"Joystick {i}: {joystick.get_name()}. It has {joystick.get_numbuttons()} buttons, {joystick.get_numaxes()} axes, and {joystick.get_numhats()} hats.")

def monitor_joystick(joystick, threshold=0.1):
    print("Monitoring joystick: Press buttons or move axes. Press Button 10 (usually L3 on PS4) to exit.")
    running = True
    while running:
        for event in pygame.event.get():  # User did something
            if event.type == pygame.JOYBUTTONDOWN:
                print(f"Button {event.button} pressed.")
                if event.button == 10:  # Adjust as necessary for your exit condition
                    running = False
            elif event.type == pygame.JOYBUTTONUP:
                print(f"Button {event.button} released.")
            elif event.type == pygame.JOYAXISMOTION:
                if abs(event.value) > threshold:  # Only print if movement exceeds threshold
                    print(f"Axis {event.axis} moved to {event.value}")
            elif event.type == pygame.JOYHATMOTION:
                print(f"Hat {event.hat} moved to {event.value}")

if __name__ == "__main__":
    initialize_pygame()
    if pygame.joystick.get_count() > 0:
        check_controllers()
        joystick = pygame.joystick.Joystick(0)
        joystick.init()
        monitor_joystick(joystick)
    else:
        print("No joystick detected.")


pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Detected 1 joystick(s)
Joystick 0: Controller (HyperX Clutch). It has 15 buttons, 6 axes, and 1 hats.
Monitoring joystick: Press buttons or move axes. Press Button 10 (usually L3 on PS4) to exit.
Hat 0 moved to (1, 0)
Hat 0 moved to (0, 0)
Button 6 pressed.
Button 6 released.
Button 8 pressed.
Button 8 released.
Button 8 pressed.
Button 8 released.
Button 8 pressed.
Button 8 released.
Button 8 pressed.
Button 8 released.
Button 8 pressed.
Button 8 released.
Button 8 pressed.
Button 8 released.
Button 8 pressed.
Button 8 released.
Axis 0 moved to -0.14844203009125034
Axis 1 moved to -0.13281655323953978
Axis 2 moved to -0.13281655323953978
Axis 0 moved to -0.21875667592394787
Axis 1 moved to -0.1640675069429609
Axis 2 moved to -0.1640675069429609
Axis 0 moved to -0.27344584490493484
Axis 1 moved to -0.2031311990722373
Axis 2 moved to -0.2031311990722373
Axis 0 moved to -0.359

KeyboardInterrupt: 